<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Meta_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install higher

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # Add this import
import torch.optim as optim
import higher
from torchvision import datasets, transforms

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 26 * 26, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # Use F for functional operations
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

# Load the dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize the model and optimizer
model = SimpleCNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Meta-learning loop
for epoch in range(5):
    for data, target in train_loader:
        data, target = data, target  # Keep data on CPU
        model.train()
        with higher.innerloop_ctx(model, optimizer) as (fmodel, diffopt):
            output = fmodel(data)
            loss = F.cross_entropy(output, target)
            diffopt.step(loss)
        # Meta-update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item()}")